# Advanced UDFs

Apache Spark&trade; and Azure Databricks&reg; allow you to create your own User Defined Functions (UDFs) specific to the needs of your data.

-sandbox
### Complex Transformations
 
UDFs provide custom, generalizable code that you can apply to ETL workloads when Spark's built-in functions won't suffice.  
In the last lesson we covered a simple version of this: UDFs that take a single DataFrame column input and return a primitive value. Often a more advanced solution is needed.

UDFs can take multiple column inputs. While UDFs cannot return multiple columns, they can return complex, named types that are easily accessible. This approach is especially helpful in ETL workloads that need to clean complex and challenging data structures.

Another other option is the new vectorized, or pandas, UDFs available in Spark 2.3. These allow for more performant UDFs written in Python.<br><br>

<div><img src="https://files.training.databricks.com/images/eLearning/ETL-Part-2/pandas-udfs.png" style="height: 400px; margin: 20px"/></div>

### UDFs with Multiple Columns

To begin making more complex UDFs, start by using multiple column inputs.  This is as simple as adding extra inputs to the function or lamda you convert to the UDF.

Run the cell below to mount the data.

In [5]:
%run "./Includes/Classroom-Setup"

Write a basic function that combines two columns.

In [7]:
def manual_add(x, y):
  return x + y

manual_add(1, 2)

Register the function as a UDF by binding it with a Python variable, adding a name to access it in the SQL API and giving it a return type.

In [9]:
from pyspark.sql.types import IntegerType

manualAddPythonUDF = spark.udf.register("manualAddSQLUDF", manual_add, IntegerType())

Create a dummy DataFrame to apply the UDF.

In [11]:
integerDF = (spark.createDataFrame([
  (1, 2),
  (3, 4),
  (5, 6)
], ["col1", "col2"]))

display(integerDF)

Apply the UDF to your DataFrame.

In [13]:
integerAddDF = integerDF.select("*", manualAddPythonUDF("col1", "col2").alias("sum"))

display(integerAddDF)

### UDFs with Complex Output

Complex outputs are helpful when you need to return multiple values from your UDF. The UDF design pattern involves returning a single column to drill down into, to pull out the desired data.

-sandbox
Start by determining the desired output.  This will look like a schema with a high level `StructType` with numerous `StructFields`.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> For a refresher on this, see the lesson **Applying Schemas to JSON Data** in ETL Part 1 module

In [16]:
from pyspark.sql.types import FloatType, StructType, StructField

mathOperationsSchema = StructType([
  StructField("sum", FloatType(), True), 
  StructField("multiplication", FloatType(), True), 
  StructField("division", FloatType(), True) 
])

Create a function that returns a tuple of your desired output.

In [18]:
def manual_math(x, y):
  return (float(x + y), float(x * y), x / float(y))

manual_math(1, 2)

Register your function as a UDF and apply it.  In this case, your return type is the schema you created.

In [20]:
manualMathPythonUDF = spark.udf.register("manualMathSQLUDF", manual_math, mathOperationsSchema)

display(integerDF.select("*", manualMathPythonUDF("col1", "col2").alias("sum")))

### Vectorized UDFs in Python

Starting in Spark 2.3, vectorized UDFs can be written in Python called Pandas UDFs.  This alleviates some of the serialization and invocation overhead of conventional Python UDFs.  While there are a number of types of these UDFs, this walkthrough focuses on scalar UDFs. This is an ideal solution for Data Scientists needing performant UDFs written in Python.

:NOTE: Your cluster will need to run Spark 2.3 in order to execute the following code.

Use the decorator syntax to designate a Pandas UDF.  The input and outputs are both Pandas series of doubles.

In [23]:
%python
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf('double', PandasUDFType.SCALAR)
def pandas_plus_one(v):
    return v + 1

Create a DataFrame to apply the UDF.

In [25]:
%python
from pyspark.sql.functions import col, rand

df = spark.range(0, 10 * 1000 * 1000)

display(df)

Apply the UDF

In [27]:
%python
display(df.withColumn('id_transformed', pandas_plus_one("id")))

%md ## Exercise 1: Multiple Column Inputs to Complex Type

Given a DataFrame of weather in various units, write a UDF that translates a column for temperature and a column for units into a complex type for temperature in three units:<br><br>

* fahrenheit
* celsius
* kelvin

### Step 1: Import and Explore the Data

Import the data sitting in `/mnt/training/weather/StationData/stationData.parquet` and save it to `weatherDF`.

In [30]:
# TODO
weatherDF = # FILL_IN

In [31]:
# TEST - Run this cell to test your solution
cols = set(weatherDF.columns)

dbTest("ET2-P-04-01-01", 2559, weatherDF.count())
dbTest("ET2-P-04-01-02", True, "TAVG" in cols and "UNIT" in cols)

print("Tests passed!")

### Step 2: Define Complex Output Type

Define the complex output type for your UDF.  This should look like the following:

| Field Name | Type |
|:-----------|:-----|
| fahrenheit | Float |
| celsius | Float |
| kelvin | Float |

In [33]:
# TODO
schema = FILL_IN

In [34]:
# TEST - Run this cell to test your solution
from pyspark.sql.types import FloatType
names = [i.name for i in schema.fields]

dbTest("ET2-P-04-02-01", 3, len(schema.fields))
dbTest("ET2-P-04-02-02", [FloatType(), FloatType(), FloatType()], [i.dataType for i in schema.fields])
dbTest("ET2-P-04-02-03", True, "fahrenheit" in names and "celsius" in names and "kelvin" in names)

print("Tests passed!")

### Step 3: Create the Function

Create a function that takes `temperature` as a Float and `unit` as a String.  `unit` will either be `F` for fahrenheit or `C` for celsius.  Return a tuple of floats of that value as `(fahrenheit, celsius, kelvin)`.  Use the following equations:

| From | To Fahrenheit | To Celsius | To Kelvin |
|:-----|:--------------|:-----------|:-----------|
| Fahrenheit | F | (F - 32) * 5/9 | (F - 32) * 5/9 + 273.15 |
| Celsius | (C * 9/5) + 32 | C | C + 273.15 |
| Kelvin | (K - 273.15) * 9/5 + 32 | K - 273.15 | K |

In [36]:
# TODO
def temperatureConverter( # FILL_IN ):

In [37]:
# TEST - Run this cell to test your solution
dbTest("ET2-P-04-03-01", (194.0, 90, 363.15), temperatureConverter(90, "C"))
dbTest("ET2-P-04-03-02", (0, -17.77777777777778, 255.3722222222222), temperatureConverter(0, "F"))

print("Tests passed!")

### Step 4: Register the UDF

Register the UDF as `temperatureConverterUDF`

In [39]:
# TODO
temperatureConverterUDF = FILL_IN

In [40]:
# TEST - Run this cell to test your solution
dbTest("ET2-P-04-04-01", (194.0, 90, 363.15), temperatureConverterUDF.func(90, "C"))
dbTest("ET2-P-04-04-02", (0, -17.77777777777778, 255.3722222222222), temperatureConverterUDF.func(0, "F"))

print("Tests passed!")

### Step 5: Apply your UDF

Create `weatherEnhancedDF` with a new column `TAVGAdjusted` that applies your UDF.

In [42]:
# TODO
weatherEnhancedDF = FILL_IN

display(weatherEnhancedDF)

In [43]:
# TEST - Run this cell to test your solution
result = weatherEnhancedDF.select("TAVGAdjusted").first()[0].asDict()

dbTest("ET2-P-04-05-01", {'fahrenheit': 61.0, 'celsius': 16.11111068725586, 'kelvin': 289.2611083984375}, result)
dbTest("ET2-P-04-05-02", 2559, weatherEnhancedDF.count())

print("Tests passed!")

## Review

**Question:** How do UDFs handle multiple column inputs and complex outputs?   
**Answer:** UDFs allow for multiple column inputs.  Complex outputs can be designated with the use of a defined schema encapsulate in a `StructType()` or a Scala case class.

**Question:** How can I do vectorized UDFs in Python and are they as performant as built-in functions?   
**Answer:** Spark 2.3 includes the use of vectorized UDFs using Pandas syntax. Even though they are vectorized, these UDFs will not be as performant built-in functions, though they will be more performant than non-vectorized Python UDFs.

## Next Steps

Start the next lesson, [Joins and Lookup Tables]($./05-Joins-and-Lookup-Tables ).

## Additional Topics & Resources

**Q:** Where can I find out more about UDFs?  
**A:** Take a look at the <a href="https://docs.azuredatabricks.net/spark/latest/spark-sql/udf-scala.html#user-defined-functions-scala" target="_blank">Databricks documentation for more details</a>

**Q:** Where can I find out more about vectorized UDFs in Python?  
**A:** Take a look at the <a href="https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html" target="_blank">Databricks blog for more details</a>

**Q:** Where can I find out more about User Defined Agregate Funcations?  
**A:** Take a look at the <a href="https://docs.azuredatabricks.net/spark/latest/spark-sql/udaf-scala.html#user-defined-aggregate-functions-scala" target="_blank">Databricks documentation for more details</a>